In [1]:
pip install awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 3.1 MB/s eta 0:00:0000:0100:01
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.20.1
    Uninstalling docutils-0.20.1:
      Successfully uninstalled docutils-0.20.1
  Attempting uninstall: botocore
    Found existing installation: botocore 1.40.0
    Uninstalling botocore-1.40.0:
      Successfully uninstalled botocore-1.40.0

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [59]:
import xarray as xr
import s3fs
import boto3
from botocore.client import Config
from botocore import UNSIGNED

s3 = boto3.client('s3',config=Config(signature_version=UNSIGNED),region_name='us-west-1')


'https://nsf-ncar-era5.s3.amazonaws.com/e5.oper.an.sfc/200001/e5.oper.an.sfc.228_131_u10n.ll025sc.2000010100_2000013123.nc'


bucket_name = 'nsf-ncar-era5'

#filename = 'e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940010100_1940010123.nc'

s3.download_file(bucket_name, 'e5.oper.an.pl/200002/e5.oper.an.pl.128_131_u.ll025uv.2000012700_2000012723.nc', 'era5_data.nc')

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [ ]:
variables= ['10u','10v','sp','2t']

In [71]:
import pandas as pd

data_range = pd.date_range(start='2000-02-01', end='2005-12-31', freq='D').strftime('%Y%m%d')

sample = 'e5.oper.an.pl/200001/e5.oper.an.pl.128_131_u.ll025uv.2000012700_2000012723.nc'

download_dft = pd.DataFrame(data_range, columns=['date'])


file_lists = [f'e5.oper.an.pl/{date[:6]}/e5.oper.an.pl.128_131_u.ll025uv.{date}00_{date}23.nc' for date in data_range]

download_dft['download_list'] = file_lists

download_dft['year'] = download_dft['date'].apply(lambda x: x[:4])

download_dft['month'] = download_dft['date'].apply(lambda x: x[4:6])

download_dft['day'] = download_dft['date'].apply(lambda x: x[6:8])

download_dft['output_filename'] = download_dft['date'].apply(
    lambda x: f'era5_pl_u_{x[:4]}_{x[4:6]}_{x[6:]}.nc'
)
download_dft

,date,download_list,year,month,day,output_filename
0,20000201,e5.oper.an.pl/200002/e5.oper.an.pl.128_131_u.l...,2000,02,01,era5_pl_u_2000_02_01.nc
1,20000202,e5.oper.an.pl/200002/e5.oper.an.pl.128_131_u.l...,2000,02,02,era5_pl_u_2000_02_02.nc
2,20000203,e5.oper.an.pl/200002/e5.oper.an.pl.128_131_u.l...,2000,02,03,era5_pl_u_2000_02_03.nc
3,20000204,e5.oper.an.pl/200002/e5.oper.an.pl.128_131_u.l...,2000,02,04,era5_pl_u_2000_02_04.nc
4,20000205,e5.oper.an.pl/200002/e5.oper.an.pl.128_131_u.l...,2000,02,05,era5_pl_u_2000_02_05.nc
...,...,...,...,...,...,...
2156,20051227,e5.oper.an.pl/200512/e5.oper.an.pl.128_131_u.l...,2005,12,27,era5_pl_u_2005_12_27.nc
2157,20051228,e5.oper.an.pl/200512/e5.oper.an.pl.128_131_u.l...,2005,12,28,era5_pl_u_2005_12_28.nc
2158,20051229,e5.oper.an.pl/200512/e5.oper.an.pl.128_131_u.l...,2005,12,29,era5_pl_u_2005_12_29.nc
2159,20051230,e5.oper.an.pl/200512/e5.oper.an.pl.128_131_u.l...,2005,12,30,era5_pl_u_2005_12_30.nc


In [72]:

def download_data(df):
    bucket_name = 'nsf-ncar-era5'
    filename = df['download_list']
    outputfile = df['output_filename']
    s3.download_file(bucket_name, filename, outputfile)

download_dft.apply(download_data,axis=1)

KeyboardInterrupt: 

<xarray.Dataset>
Dimensions:    (time: 24, level: 37, latitude: 721, longitude: 1440)
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level      (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 2000-01-01 ... 2000-01-01T23:00:00
Data variables:
    PV         (time, level, latitude, longitude) float32 ...
    utc_date   (time) int32 ...
Attributes:
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB 1 data to netC...
    NETCDF_VERSION:       4.6.1
    CONVERSION_PLATFORM:  Linux casper10 3.10.0-693.21.1.el7.x86_64 #1 SMP We...
    CONVERSION_DATE:      Fri Jul 26 11:06:55 MDT 2019
    Conventions:          CF-1.6
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    history:              Fri Jul 26 11:07:33 2019: ncks -4 --ppc default=7 e...
    NCO:                  netCDF Operators version 4.7.4 (http://nco.sf.net)

In [15]:
import boto3

# Initialize the S3 client without credentials
s3 = boto3.client('s3', config=boto3.session.Config(signature_version=UNSIGNED))

bucket_name = 'nsf-ncar-era5'
prefix = '' # Use an empty string to list all objects in the root of the bucket

try:
    # List objects in the specified bucket and prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    if 'Contents' in response:
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print(f"No objects found in s3://{bucket_name}/.")

except Exception as e:
    print(f"An error occurred: {e}")

e5.oper.an.pl/
e5.oper.an.pl/194001/
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940010100_1940010123.nc
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940010200_1940010223.nc
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940010300_1940010323.nc
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940010400_1940010423.nc
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940010500_1940010523.nc
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940010600_1940010623.nc
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940010700_1940010723.nc
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940010800_1940010823.nc
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940010900_1940010923.nc
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940011000_1940011023.nc
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940011100_1940011123.nc
e5.oper.an.pl/194001/e5.oper.an.pl.128_060_pv.ll025sc.1940011200_1940011223.nc
e5.oper.an.pl/1